<a href="https://colab.research.google.com/github/subhabrata-ghosh-1988/Transformers/blob/main/Transformers_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import os
import random
import torch
import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
from keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
!pip install transformers
import transformers


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def set_seed(seed):
        # For reproducibility
        os.environ['PYTHONHASHSEED']=str(seed)
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [7]:
set_seed(42)

In [8]:
df = pd.read_json("https://raw.githubusercontent.com/subhabrata-ghosh-1988/Transformers/main/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [9]:
df.drop('article_link', axis=1, inplace=True)

In [10]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [11]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['headline']=df['headline'].apply(denoise_text)

In [12]:
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.headline)
corpus[:5]

['thirtysomething', 'scientists', 'unveil', 'doomsday', 'clock']

In [13]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

{'area': 494,
 'day': 435,
 'donald': 472,
 'like': 428,
 'man': 1351,
 'new': 1637,
 'one': 527,
 'says': 482,
 'trump': 1284,
 'woman': 483}

In [14]:
X = df['headline']
y = df['is_sarcastic']

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0 , stratify = y)

In [16]:
from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [17]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=400):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [18]:
x_train = fast_encode(X_train.values, fast_tokenizer, maxlen=400)
x_test = fast_encode(X_test.values, fast_tokenizer, maxlen=400)

In [19]:
from keras.models import model_from_json
from transformers import BertTokenizer, TFBertModel
!pip install gdown
import gdown

In [20]:
#gdown https://drive.google.com/uc?id=15VXuCEgsHdJ09ZCgAFfJzHYMWzti1bDC
#gdown https://drive.google.com/uc?id=1ts6wzpqycSCab92xK8PR8-BOHLrwesCi
gdown.download("https://drive.google.com/uc?id=1ts6wzpqycSCab92xK8PR8-BOHLrwesCi")
gdown.download("https://drive.google.com/uc?id=15VXuCEgsHdJ09ZCgAFfJzHYMWzti1bDC")

Downloading...
From: https://drive.google.com/uc?id=1ts6wzpqycSCab92xK8PR8-BOHLrwesCi
To: /content/bert-model.json
100%|██████████| 3.13k/3.13k [00:00<00:00, 5.63MB/s]
Downloading...
From: https://drive.google.com/uc?id=15VXuCEgsHdJ09ZCgAFfJzHYMWzti1bDC
To: /content/bert-model.h5
100%|██████████| 438M/438M [00:12<00:00, 36.5MB/s]


'bert-model.h5'

In [21]:
# load json and create model
json_file = open('bert-model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_bert = model_from_json(loaded_model_json,custom_objects={'TFBertModel': TFBertModel})
# load weights into new model
loaded_model_bert.load_weights("bert-model.h5")
print("Loaded model from drive")

Loaded model from drive


In [22]:
loaded_model_bert.compile(Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    

In [23]:
print("Accuracy of the model on Testing Data is - " , loaded_model_bert.evaluate(x_test,y_test)[1]*100 , "%")

224/224 [==============================] - 214s 929ms/step - loss: 0.4553 - accuracy: 0.8422
Accuracy of the model on Testing Data is -  84.22082662582397 %


In [26]:
bert_uncased_pred = loaded_model_bert.predict(x_test)

In [27]:
gdown.download("https://drive.google.com/uc?id=1Zz8u68-3I7_WZ3puMuHW7Gb7aYXy0icJ")
gdown.download("https://drive.google.com/uc?id=1FIeuaCmkSGKMVTUWyaHe25Zp3--6ToL5")

Downloading...
From: https://drive.google.com/uc?id=1Zz8u68-3I7_WZ3puMuHW7Gb7aYXy0icJ
To: /content/distil-bert-model.h5
100%|██████████| 266M/266M [00:09<00:00, 29.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FIeuaCmkSGKMVTUWyaHe25Zp3--6ToL5
To: /content/distil-bert-model.json
100%|██████████| 3.04k/3.04k [00:00<00:00, 5.52MB/s]


'distil-bert-model.json'

In [28]:
from keras.models import model_from_json
from transformers import BertTokenizer, TFBertModel,TFDistilBertModel

In [29]:
# load json and create model
json_file = open('distil-bert-model.json', 'r')
distil_bert_loaded_model_json = json_file.read()
json_file.close()
distil_bert_loaded_model = model_from_json(distil_bert_loaded_model_json,custom_objects={'TFDistilBertModel': TFDistilBertModel})
# load weights into new model
distil_bert_loaded_model.load_weights("distil-bert-model.h5")
print("Loaded model from disk")

Loaded model from disk


In [30]:
distil_bert_loaded_model.compile(Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    

In [32]:
print("Accuracy of the model on Testing Data is - " , distil_bert_loaded_model.evaluate(x_test,y_test)[1]*100 , "%")

224/224 [==============================] - 102s 446ms/step - loss: 0.4233 - accuracy: 0.8341
Accuracy of the model on Testing Data is -  83.41020345687866 %


In [33]:
distil_bert_uncased_pred=distil_bert_loaded_model.predict(x_test)

In [48]:
gdown.download("https://drive.google.com/uc?id=1osFck55L-RPCqH1CYoJKmWtwY8ysM5nR")
gdown.download("https://drive.google.com/uc?id=17NtzDA4dvMNCcxx1njvDBP8fcP65YMnx")

Downloading...
From: https://drive.google.com/uc?id=1osFck55L-RPCqH1CYoJKmWtwY8ysM5nR
To: /content/roberta-model.json
100%|██████████| 4.17k/4.17k [00:00<00:00, 2.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=17NtzDA4dvMNCcxx1njvDBP8fcP65YMnx
To: /content/roberta-model.h5
100%|██████████| 729M/729M [00:19<00:00, 36.8MB/s]


'roberta-model.h5'

In [56]:
from keras.models import model_from_json
from transformers import BertTokenizer, TFBertModel,TFRobertaModel

In [68]:
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [69]:
MODEL_NAME = 'roberta-base'
# MAX_LEN= 256
MAX_LEN= 256
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 8 * strategy.num_replicas_in_sync
EPOCHS = 3

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

In [70]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [71]:
# load json and create model
json_file = open('roberta-model.json', 'r')
roberta_loaded_model_json = json_file.read()
json_file.close()
roberta_loaded_model = model_from_json(roberta_loaded_model_json,custom_objects={'TFRobertaModel': TFRobertaModel})
# load weights into new model
roberta_loaded_model.load_weights("roberta-model.h5")
print("Loaded model from disk")

Loaded model from disk


In [72]:
roberta_loaded_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [73]:
X_train = roberta_encode(X_train, tokenizer)
X_test = roberta_encode(X_test, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [ ]:
print("Accuracy of the model on Testing Data is - " , roberta_loaded_model.evaluate(X_test, y_test, verbose=0)[1]*100 , "%")

In [59]:
#print("Accuracy of the model on Testing Data is - " , roberta_loaded_model.evaluate(X_test,y_test)[1]*100 , "%")

ValueError: ignored

In [41]:
mean_pred = np.mean([bert_uncased_pred, distil_bert_uncased_pred], axis=0)

In [42]:
mean_pred

array([[0.999745  ],
       [0.995837  ],
       [0.03260009],
       ...,
       [0.8229078 ],
       [0.46866944],
       [0.172309  ]], dtype=float32)

In [43]:
mean_pred = np.round(mean_pred).astype(int)

In [47]:
print(classification_report(y_test, mean_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      3746
           1       0.86      0.82      0.84      3409

    accuracy                           0.85      7155
   macro avg       0.85      0.85      0.85      7155
weighted avg       0.85      0.85      0.85      7155

